<a href="https://colab.research.google.com/github/lizardnote/Text-Analytics/blob/main/LLama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라마3 토크나이저
공개된 LLM 불러오기 위한 transformer 패키지 사용

In [1]:
from transformers import AutoTokenizer

In [2]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

- encode() 는 사람의 입력을 LLM이 이해할 수 있도록 정수 인코딩 하는 과정
- decode() 는 다시 사람의 입력으로 해석하게 바꿔줌

In [3]:
tokenizer.encode("안녕하세요 반가워요")

[128000, 101193, 124409, 64857, 20565, 103430, 36811]

In [4]:
tokenizer.decode([128000, 101193, 124409, 64857, 20565, 103430, 36811])

'<|begin_of_text|>안녕하세요 반가워요'

- 단, 여기서 쓰인 토크나이저는 반드시 해당 모델을 사용할 때만 쓸 수 있다.
- 모델마다 어떤 토큰에 정수가 맵핑 되어있는지 다르기 때문

## 주의
예를 들어서 A라는 LLM이 있다고 가정해봅시다. 학습 시에 아래와 같은 형식으로 학습이 되었습니다.
```
user: 안녕~ 반가워!
assistant: 네, 반갑습니다.
```

그런데 실제로 사용할 때는 아래 형식으로 사용한다고 가정해봅시다.
```
<user_question> 안녕~ 반가워! </user_question>
<answer> 네, 반갑습니다. </answer>
```
이 경우 학습할 때의 프롬프트 형식과 실제 사용할 때의 프롬프트 형식이 다르기 때문에 LLM의 성능 저하가 발생할 수 있습니다.

따라서 학습 시의 형식을 따라서 프롬프트 호출을 해야만 합니다. 그렇다면 우리가 사용하고자 하는 LLM의 학습 시의 형식은 어떻게 알까요? 보통 LLM을 공개할 때 같이 공개하는 편입니다. 하지만 이를 자동으로 처리하는 방법도 있습니다. 바로 지금부터 배울 허깅페이스의 토크나이저를 활용하는 방법입니다.



In [5]:
messages = [
    {"role": "user", "content": "안녕하세요 반가워요"},
    {"role": "assistant", "content": "네, 반갑습니다"}
]

In [6]:
template_message = tokenizer.apply_chat_template(messages, tokenize=False)
print(template_message)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

안녕하세요 반가워요<|eot_id|><|start_header_id|>assistant<|end_header_id|>

네, 반갑습니다<|eot_id|>


In [7]:
encodeds = tokenizer.apply_chat_template(messages) #tokenize = True 디폴트

In [8]:
print('템플릿 적용 및 정수 인코딩 후:\n', encodeds)
print('--' * 100)
print('템플릿 적용 및 정수 인코딩 결과를 복원:\n',tokenizer.decode((encodeds)))

템플릿 적용 및 정수 인코딩 후:
 [128000, 128006, 882, 128007, 271, 101193, 124409, 64857, 20565, 103430, 36811, 128009, 128006, 78191, 128007, 271, 101886, 11, 64857, 115072, 39331, 128009]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
템플릿 적용 및 정수 인코딩 결과를 복원:
 <|begin_of_text|><|start_header_id|>user<|end_header_id|>

안녕하세요 반가워요<|eot_id|><|start_header_id|>assistant<|end_header_id|>

네, 반갑습니다<|eot_id|>


정수 인코딩 후 다시 복원(디코드) 한 결과는 위처럼 출력된다.

## 어시스턴트 시작점

add_generation_prompt=True를 사용하면 LLM이 답변을 하기 바로 직전의 프롬프트까지 추가한다.  
LLama3의 경우, `<|start_header_id|>assistant<|end_header_id|>`가 추가되는 것을 확인할 수 있다.

In [9]:
encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
print(encodeds)
print('템플릿 적용 및 정수 인코딩 결과를 복원:\n',tokenizer.decode((encodeds)))

[128000, 128006, 882, 128007, 271, 101193, 124409, 64857, 20565, 103430, 36811, 128009, 128006, 78191, 128007, 271, 101886, 11, 64857, 115072, 39331, 128009, 128006, 78191, 128007, 271]
템플릿 적용 및 정수 인코딩 결과를 복원:
 <|begin_of_text|><|start_header_id|>user<|end_header_id|>

안녕하세요 반가워요<|eot_id|><|start_header_id|>assistant<|end_header_id|>

네, 반갑습니다<|eot_id|><|start_header_id|>assistant<|end_header_id|>




**시스템 프롬프트**
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 패스트캠퍼스 챗봇 패캠이입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|>
```

**유저 프롬프트**  
```
<|start_header_id|>user<|end_header_id|>

당신이 누군지 설명해주십시오<|eot_id|>
```

**어시스턴트 시작점**
```
<|start_header_id|>assistant<|end_header_id|>
```

모두 결합하면?
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 패스트캠퍼스 챗봇 패캠이입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

당신이 누군지 설명해주십시오<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```

## Qwen2 템플릿 확인하기
모델마다 템플릿이 전혀 다르다. LLama3 대신 Qwen2를 확인해보자

In [10]:
model_id = "Qwen/Qwen2-7B-Instruct"
qwen_tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
messages = [
    {"role": "system", "content": "당신은 패스트캠퍼스 챗봇 패캠이입니다. 묻는 말에 친절하고 정확하게 답변하세요."},
    {"role": "user", "content": "당신이 누군지 설명해주십시오"}
]

encodeds = qwen_tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
model_inputs = encodeds.to(device)

In [15]:
print(qwen_tokenizer.decode(model_inputs[0]))

<|im_start|>system
당신은 패스트캠퍼스 챗봇 패캠이입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|im_end|>
<|im_start|>user
당신이 누군지 설명해주십시오<|im_end|>
<|im_start|>assistant

